In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from glob import glob
import seaborn as sns

In [4]:
def compute_matches(dir_path, mode, epoch, body_part):
    labels_df = pd.read_csv(fr"{dir_path}/files/saved_preds/{mode}_labels.csv")
    preds_df = pd.read_csv(fr"{dir_path}/files/saved_preds/{mode}_preds.csv")
    
    if mode == 'val':
        wanted_labels_keys = list(labels_df.filter(regex=body_part).filter(regex='val').keys())
        wanted_preds_keys = list(preds_df.filter(regex=body_part).filter(regex='val').keys())
    else:
        wanted_labels_keys = list(labels_df.filter(regex=body_part).keys())
        wanted_preds_keys = list(preds_df.filter(regex=body_part).keys())

    temp = labels_df[wanted_labels_keys]
    labels_df = pd.DataFrame(temp.iloc[-1]).rename(columns={epoch: 'Labels'})

    temp = preds_df[wanted_preds_keys]
    preds_df = pd.DataFrame(temp.iloc[-1]).rename(columns={epoch: 'Preds'})
    
    concat_df = pd.concat([labels_df, preds_df], axis=1)
    concat_df['Rounded preds'] = np.round(concat_df['Preds'])
    concat_df['Ceiled']=concat_df['Rounded preds'].apply(lambda x: 3.0 if x > 3.0 else x)
    concat_df['Diff'] = np.absolute(concat_df['Ceiled'] - concat_df['Labels'])
    
    print(concat_df['Diff'].value_counts())
    
    

In [ ]:
dir_path = r"/home/pedke/multi-task-learning/wandb/run-20230823_144508-lk18695h"
mode = 'train'
epoch = 99
body_part = 'Glans'

compute_matches(dir_path, mode, epoch, body_part)